신경망 학습
========
학습을 해보자! 학습을 하려면, 제일 먼저 지금 학습이 얼마나 잘 되었는지를 수치로 평가해주는 **손실 함수**<small>loss function</small>가 필요하다. 많이 쓰이는 손실 함수 두개를 직접 구현해보자.

### 평균 제곱 오차 <small>Mean squared error, MSE</small>
$${\displaystyle 
{\rm MSE}(y, t) = \frac1n \sum_k^n (y_k - t_k)^2
}$$

In [1]:
def mean_squared_error(expected, actual):
    return ((expected - actual)**2).mean(axis=0)

### 교차 엔트로피 오차 <small>Cross entropy error, CEE</small>
$${\displaystyle 
{\rm CEE}(y, t) = - \sum_k t_k {\rm log}_2 y_k
}$$

In [2]:
def cross_entropy_error(expected, actual):
    epsilon = 1E-7
    return -np.sum(actual * np.log(expected + epsilon), axis=0) 

### 미니배치 학습
교차 엔트로피 오차 수식을 살짝만 바꿔서, 미니배치용 손실함수를 만들어서 쓰면 된다.

$${\displaystyle 
{\rm CEE}(y, t) = - \frac1N \sum_i^N \sum_k t_{ik} {\rm log}_2 y_{ik}
}$$

In [3]:
import numpy as np
import mnist

# Load training images
MNIST = mnist.load()
train_img = MNIST['train_img']
train_label = MNIST['train_label']

# Randomly sample 10 images from the training set
batch_size = 10
subset = np.random.choice(train_img.shape[0], batch_size)
batch_img = train_img[subset]
batch_label = train_label[subset]

# Cross entropy error function for batch input
def cross_entropy_error(expected, actual):
    if expected.ndim == 1:
        expected = expected.reshape(1, expected.size)
        actual = actual.reshape(1, actual.size)
    
    batch_size = expected.shape[0]
    return -np.sum(actual * np.log(expected), axis=0) / batch_size

### 수치 미분 <small>Numerical differentiation</small>
중앙 차분법을 쓸것이다.

$${\displaystyle 
f'(x) \simeq \frac{f(x+h) - f(x-h)}{2h} \text{ where } h \text{ is small enough}
}$$

In [28]:
def derivate(f, h=1E-4):
    """
    미분 연산자. 이 함수의 파라미터로 미분하고싶은 함수를 넘기면, 도함수가 결과로 나온다.

    .. code-block:: python

       linear = lambda x: x**2 + 2*x + 1
       deriv = derivate(linear)

       deriv(0) # 2
       deriv(2) # 6
    """
    return lambda x: (f(x+h) - f(x-h))/(2*h)

func = lambda x: 0.01 * x**2 + 0.1 * x
deriv = derivate(func)
deriv_analytic = lambda x: 0.02 * x + 0.1

for x in 5, 10, 15:
    print(f'해석적 미분 : {deriv_analytic(x)}')
    print(f'수치 미분   : {deriv(x)}')
    print()

해석적 미분 : 0.2
수치 미분   : 0.1999999999990898

해석적 미분 : 0.30000000000000004
수치 미분   : 0.2999999999986347

해석적 미분 : 0.4
수치 미분   : 0.4000000000026205



### 편미분
걍 하면 된다

$${\displaystyle 
\begin{align}
\frac{\partial f}{\partial x_i}(a_1,\ldots,a_n) \simeq & \frac{f(a_1,\ldots,a_i+h,\ldots,a_n) - f(a_1,\ldots, a_i-h, \dots,a_n)}{2h}
\\ & \text{ where } h \text{ is small enough}
\end{align}
}$$

In [29]:
def partial_derivative(f, nth, h=1E-4):
    """
    편미분 연산자. 이 함수의 파라미터로 편미분하고싶은 함수와, 몇번째 파라미터를 기준으로
    편미분할지 전달하면, 편도함수가 결과로 나온다.

    .. code-block:: python

       surface = lambda a, b: a**2 + b**2
       partial = partial_derivative(surface, 0)

       partial(1, 2) # 2
       partial(2, 3) # 4
    """
    def func(*args):
        param1, param2 = list(args), list(args)
        param1[nth] += h
        param2[nth] -= h
        return (f(*param1) - f(*param2))/(2*h)
    return func

func = lambda a, b, c: a**2 + b**2 + c**2
partial = partial_derivative(func, 0)
partial_analytic = lambda a, b, c: 2*a

for args in (1, 2, 3), (4, 5, 6), (7, 8, 9):
    print(f'해석적 편미분 : {partial_analytic(*args)}')
    print(f'수치 편미분   : {partial(*args)}')
    print()

해석적 편미분 : 2
수치 편미분   : 1.9999999999953388

해석적 편미분 : 8
수치 편미분   : 8.00000000005241

해석적 편미분 : 14
수치 편미분   : 13.999999999896318



### 그래디언트 <small>Gradient</small>
각 성분에 대한 편미분으로 구성된 열벡터이다.

$${\displaystyle
\nabla f=\left({\frac {\partial f}{\partial x_{1}}},\dots ,{\frac {\partial f}{\partial x_{n}}}\right)
}$$

In [60]:
import inspect

def grad(f, h=1E-4):
    """
    델(∇) 연산자. nabla 연산자의 인자로 스칼라 함수를 넣으면, 그 함수의 그래디언트가 반환된다.
    
    .. code-block:: python
    
       surface = lambda a, b: a**2 + b**2
       gradient = grad(surface)
       
       gradient(1, 2) # (2, 4)
       gradient(2, 3) # (4, 6)
    """
    derivs = tuple(partial_derivative(func, i) for i in range(len(inspect.signature(func).parameters)))
    return lambda *args: tuple(deriv(*args) for deriv in derivs)
    
func = lambda a, b, c: a**2 + b**2 + c**2
gradient = grad(func)
gradient_analytic = lambda a, b, c: (2*a, 2*b, 2*c)

for args in (1, 2, 3), (4, 5, 6), (7, 8, 9):
    print(f'해석적 그래디언트 : {gradient_analytic(*args)}')
    print(f'수치 그래디언트   : {gradient(*args)}')
    print()

해석적 그래디언트 : (2, 4, 6)
수치 그래디언트   : (1.9999999999953388, 3.9999999999995595, 6.000000000012662)

해석적 그래디언트 : (8, 10, 12)
수치 그래디언트   : (8.00000000005241, 9.999999999976694, 11.999999999972033)

해석적 그래디언트 : (14, 16, 18)
수치 그래디언트   : (13.999999999896318, 15.99999999996271, 18.000000000029104)



###### References
- https://www.codecogs.com/latex/eqneditor.php